<h1> [Method 1] Stage 4 Dựng mô hình RNN và huấn luyện </h1>

__9416  9440 là 2 article_id bị trùng__

Hệ thống lại các stage:

- Stage 1: Segmentation bằng thư viện VNCoreNLP
    - segmented_corpus.json: Law corpus đã được segmentation
    - train_ques_segmented.json: Câu hỏi đã được segmentation
    - article_id.json:  (law_id, article_id) tương ứng với list trong file `segmented_corpus.json`
    - relevance_pair_idx: cặp id (question_id, article_id) liên quan đến nhau. ID là id của list `segmented_corpus.json` và `train_ques_segmented.json`
    - train_test_split: thư mục lưu id của question dùng để huấn luyện và test (cố định)

- Stage 2: Sử dụng PhoBERT để lấy token embedding

- Stage 3: Sử dụng BM25 để ranking document

- Stage 3a: Dựng cặp question-article dựa vào kết quả ranking

- Stage 4: Dựng bộ phân lớp bằng (XGBoost / RNN, CNN, ...)

In [ ]:
# ! gdown  https://drive.google.com/uc?id=1JfCUMRnvkLw2Y2fJwHbW5jjwNbz_85aQ
# ! unzip zaloai.zip

In [1]:
import json
import os
import torch
# from transformers import AutoModel, AutoTokenizer
import numpy as np
from tqdm.notebook import tqdm
import random
import itertools
from torch.utils.data import Dataset, DataLoader
from torch import nn
import os
# from torchsummary import summary
import xgboost as xgb
from sklearn.model_selection import train_test_split, StratifiedKFold
import fasttext
import gc
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import glob


device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [2]:
! nvidia-smi

Sat Dec 11 09:52:40 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.44       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P0    27W / 250W |      2MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
# phobert = AutoModel.from_pretrained("/content/drive/MyDrive/Model/phobert-base").to(device)
# phobert.eval()
# tokenizer = AutoTokenizer.from_pretrained("/content/drive/MyDrive/Model/phobert-tokenizer")

# Đọc dữ liệu

In [3]:
# ROOT_DIR = '/content/drive/MyDrive/Datasets/ZaloAIChallenge2021'
# ROOT_DIR = './'
ROOT_DIR = '/content/drive/MyDrive/ZaloAIChallenge2021'


CORPUS_PATH = os.path.join(ROOT_DIR, 'zac2021-ltr-data/legal_corpus.json')
TRAIN_QUES_PATH = os.path.join(ROOT_DIR, 'zac2021-ltr-data/train_question_answer.json')
PREPROC_STAGE1 = os.path.join(ROOT_DIR, 'segmented_corpus.json')
PREPROC_STAGE1_QUES = os.path.join(ROOT_DIR, 'train_ques_segmented.json')
ARTICLE_ID = os.path.join(ROOT_DIR, 'article_id.json')
PAIR_IDX_PATH =  os.path.join(ROOT_DIR, 'relevance_pair_idx.json')
TRAIN_IDX_PATH = os.path.join(ROOT_DIR, 'train_test_split/train_idx.json')
TEST_IDX_PATH = os.path.join(ROOT_DIR, 'train_test_split/test_idx.json')
RELEVANCE_PAIR_PATH = os.path.join(ROOT_DIR, 'relevance_pair_idx.json')

with open(CORPUS_PATH, 'r') as f:
    competition_corpus = json.load(f)

with open(TRAIN_QUES_PATH, 'r') as f:
    competition_ques = json.load(f)

with open(ARTICLE_ID, 'r') as f:
    article_idx = json.load(f)

with open(PREPROC_STAGE1, 'r') as f:
    preproc_stage1 = json.load(f)

with open(PAIR_IDX_PATH, 'r') as f:
    list_pair_idx = json.load(f)

with open(TRAIN_IDX_PATH, 'r') as f:
    idx_train = json.load(f)

with open(TEST_IDX_PATH, 'r') as f:
    idx_test = json.load(f)

with open(PREPROC_STAGE1_QUES, 'r') as f:
    preproc_stage1_ques = json.load(f)

with open(RELEVANCE_PAIR_PATH, 'r') as f:
    relevance_pair = json.load(f)


articles = [iarticle  for icorpus in competition_corpus for iarticle in icorpus.get('articles')]
ques = [iques for iques in competition_ques.get('items')]

In [13]:
! gdown https://drive.google.com/uc?id=1gf8d5T_E8GLDX7LHrmxKkBmzEWOQhqEF
with open('segmented_title.json', 'r') as f:
    preproc_stage1_title = json.load(f)

Downloading...
From: https://drive.google.com/uc?id=1gf8d5T_E8GLDX7LHrmxKkBmzEWOQhqEF
To: /content/segmented_title.json
100% 10.2M/10.2M [00:00<00:00, 89.6MB/s]


In [ ]:
ROOT_DIR = '/content/drive/MyDrive/ZaloAIChallenge2021'
TRAIN_IDX_PATH = os.path.join(ROOT_DIR, 'train_test_split/train_idx.json')
RELEVANCE_PAIR_PATH = os.path.join(ROOT_DIR, 'relevance_pair_idx.json')

with open(TRAIN_IDX_PATH, 'r') as f:
    idx_train = json.load(f)
with open(RELEVANCE_PAIR_PATH, 'r') as f:
    relevance_pair = json.load(f)

In [4]:
# Dọn dẹp bộ nhớ
articles = None
ques = None
competition_corpus = None
competition_ques = None
gc.collect()

42

# Xây dựng TF-IDF vectorizer

In [4]:
uni_tfidf_vrizer = TfidfVectorizer()
uni_corpus = [
    ' '.join([' '.join(sent) for sent in arti]) for arti in preproc_stage1
]
unigram_tfidf_corpus = uni_tfidf_vrizer.fit_transform(uni_corpus)

In [5]:
def make_bi_from_sent(sent):
    return [f'{sent[i]}_{sent[i+1]}' for i in range(len(sent)-1)]

bi_tfidf_vrizer = TfidfVectorizer()
bi_corpus = [
             ' '.join([' '.join(make_bi_from_sent(sent)) for sent in arti])
             for arti in preproc_stage1
]
bigram_tfidf_corpus = bi_tfidf_vrizer.fit_transform(bi_corpus)

# Chia tập câu hỏi training ra làm tập c_train và tập dev (để chọn ngưỡng)

In [6]:
dev_percent = 0.2
_split = int(len(idx_train) * 0.2)
idx_dev = idx_train[-_split:]
idx_ctrain = idx_train[:-_split]
len(idx_ctrain), len(idx_dev)

(2045, 511)

# Hàm đọc kết quả ranking

In [ ]:
! gdown https://drive.google.com/uc?id=1sbKkQtWTx-h6HCm5z0ETfrOob6DDz5a5
! unzip /content/rankingtop200.zip

In [8]:
RANKING_DIR = '/content/top200'
# RANKING_DIR = '/content/drive/MyDrive/ZaloAIChallenge2021/ranking-result/top1000'
def get_ranking_result(top_n, qid):
#   SAVE_TOPN_DIR = f'{ROOT_DIR}/ranking-result/top{top_n}'
  with open(os.path.join(RANKING_DIR, f'ranking-idx{qid}.npy'), 'rb') as f:
     return np.load(f)


# Đọc chỉ số các cặp question - article - label



In [9]:
%%capture
# ! gdown https://drive.google.com/uc?id=1jmD6EqLagBtjmTkIHDbW2Ru1wXDIMH3U
! gdown https://drive.google.com/uc?id=17AGdPW6YBOV3yyE9tgR1SZ1FQ1DJga-c

In [15]:
build = True
topn = 200

if build:
    all_ques_art_pair = []
    for qid in tqdm(idx_ctrain):
        set_aid = set()

        rel_aid = [_aid for (_qid, _aid) in relevance_pair if _qid == qid]
        set_aid.update(rel_aid)

        ranking = get_ranking_result(top_n=200, qid=qid)[-topn:]
        set_aid.update(ranking)

        all_ques_art_pair += [(qid, aid, int(aid in rel_aid)) for aid in set_aid]

    with open(f'ques_art_pair_ctrain_{topn}.npy', 'wb') as f:
        np.save(f, np.array(all_ques_art_pair))
else:
    with open(os.path.join(f'ques_art_pair_ctrain_{topn}.npy' ), 'rb') as f:
      all_ques_art_pair = np.load(f)
    print(len(all_ques_art_pair))

  0%|          | 0/2045 [00:00<?, ?it/s]

In [16]:
a = set()
for qid, aid, label in all_ques_art_pair:
    if qid not in idx_ctrain:
        a.add(qid)
len(a)

0

# Sử dụng pp học máy để huấn luyện

Để sử dụng pp học máy hiệu quả thì cũng cần một bộ embedding word để lấy được feature. 

Chạy PhoBERT để lấy embeđing ?? 

## Xây dựng feature fast-text embedding

In [ ]:
%%capture
! gdown https://drive.google.com/uc?id=1-81eLndmPLR7Ug9uMs9fz8GnV23i41LQ

In [17]:
! gdown https://drive.google.com/uc?id=1kh5VXxKyc9HJFi4CwD5s7UvL6XgCrpwP

Downloading...
From: https://drive.google.com/uc?id=1kh5VXxKyc9HJFi4CwD5s7UvL6XgCrpwP
To: /content/vnlaw_ft.bin
100% 844M/844M [00:03<00:00, 243MB/s]  


In [ ]:
unigram_tfidf_corpus.

In [18]:
GET_TOKEN_EMBED = 0
GET_SENT_EMBED = 1
# model = fasttext.load_model('wiki.vi.bin')
model = fasttext.load_model('vnlaw_ft.bin')


def display_segmented_corpus(l):
  print(' '.join([k for i in l for k in i]))


def get_embed_article(aid):    
    inp_ids =   np.array([
                 model.get_word_vector(token) 
                 for sent in preproc_stage1[aid]
                 for token in sent
                ]) # tokens * 300
    return inp_ids.mean(axis = 0) # (300)

def get_embed_question(qid):
    inp_ids = np.array([
                        model.get_word_vector(token)
                        for sent in preproc_stage1_ques[qid]
                        for token in sent
    ])
    return inp_ids.mean(axis=0)

def get_embed_title(aid):
  inp_ids = np.array([
    model.get_word_vector(tok)
    for sent in preproc_stage1_title
    for tok in sent
  ])

  



np.concatenate((get_embed_article(0), get_embed_question(0))).shape

(200,)

In [30]:
# np.sum(np.array([len(title) for title in preproc_stage1_title]) == 3)

In [ ]:
# ! gdown https://drive.google.com/uc?id=1-3EBfqI4lbTllcAkc8MZ1tojriY-cJmm
! gdown https://drive.google.com/uc?id=1-0Wbx_EYSKR9Ehej2aNj6apldASiGjyg

In [ ]:
build = True
print(topn)
ftembed_ques = []
ftembed_arti = []
ftembed_title = []

if build:
    X = np.array([
        np.concatenate((get_embed_question(qid), get_embed_article(aid)))
        for (qid, aid, label) in tqdm(all_ques_art_pair)
    ])
    with open(f'fast-text-ctrain-{topn}.npy', 'wb') as f:
        np.save(f, X)
else:
    with open(f'fast-text-ctrain-{topn}.npy', 'rb') as f:
        X = np.load(f)

200


  0%|          | 0/409173 [00:00<?, ?it/s]

In [ ]:
y = np.array([label for (qid, aid, label) in tqdm(all_ques_art_pair) ])

  0%|          | 0/409173 [00:00<?, ?it/s]

# Thêm feature tfidf (uni và bi)

In [ ]:
# Xây dựng mảng tfidf của question
uni_ques = [
            ' '.join([tok for sent in ques for tok in sent])
            for ques in preproc_stage1_ques
]
uni_tfidf_ques = uni_tfidf_vrizer.transform(uni_ques)

bi_ques = [
           ' '.join(make_bi_from_sent([tok for sent in ques for tok in sent]))
           for ques in preproc_stage1_ques
]
bi_tfidf_ques = bi_tfidf_vrizer.transform(bi_ques)

In [ ]:
uni_cosine = cosine_similarity(uni_tfidf_ques, unigram_tfidf_corpus, dense_output=False)
bi_cosine = cosine_similarity(bi_tfidf_ques, bigram_tfidf_corpus, dense_output=False)

In [ ]:
def calculate_tfidf_feature(qid, aid):
    uni_sim = uni_cosine[qid, aid]
    bi_sim = bi_cosine[qid, aid]
    return np.array([uni_sim, bi_sim])


tfidf_feature = np.array([
                 calculate_tfidf_feature(qid, aid) for (qid, aid, label) in tqdm(all_ques_art_pair)
])

  0%|          | 0/409173 [00:00<?, ?it/s]

In [ ]:
X = np.hstack((X, tfidf_feature))

In [ ]:
with open('x_feature.npy', 'wb') as f:
    np.save(f, X)

## Huấn luyện mô hình XGBoost

In [ ]:
with open('x_feature.npy', 'rb') as f:
    X = np.load(f)

In [ ]:
# # Reset
# list_clf = None

def base_xgb_model():
  return xgb.XGBClassifier(
          n_estimators=5000,
          max_depth=12,
          learning_rate=0.02,
          subsample=0.8,
          colsample_bytree=0.4,
          missing=-1,
          eval_metric='auc',
          # USE CPU
          # nthread=4,
          # tree_method='hist'
          # USE GPU
          tree_method='gpu_hist' 
      )

In [ ]:
X.shape, y.shape

((409173, 202), (409173,))

In [ ]:
use_kfolds = True
list_clf = []

if use_kfolds:
    n_folds = 6

    skf = StratifiedKFold(n_splits=n_folds, random_state=42, shuffle=True)

    for train_index, test_index in skf.split(X, y):
        clf = base_xgb_model()
        clf.fit(X[train_index], y[train_index], 
                eval_set=[(X[test_index], y[test_index])],
                verbose=100, early_stopping_rounds=200)
        
        list_clf.append(clf)
else:
    clf = base_xgb_model()
    X_train, X_test, y_train , y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    clf.fit(X_train, y_train,
            eval_set=[(X_test, y_test)],
            verbose=100, early_stopping_rounds=200)
    list_clf.append(clf)

[0]	validation_0-auc:0.548851
Will train until validation_0-auc hasn't improved in 200 rounds.
[100]	validation_0-auc:0.896376
[200]	validation_0-auc:0.916209
[300]	validation_0-auc:0.928305
[400]	validation_0-auc:0.934683
[500]	validation_0-auc:0.934571
[600]	validation_0-auc:0.935092
Stopping. Best iteration:
[420]	validation_0-auc:0.935306

[0]	validation_0-auc:0.545733
Will train until validation_0-auc hasn't improved in 200 rounds.
[100]	validation_0-auc:0.899953
[200]	validation_0-auc:0.914394
[300]	validation_0-auc:0.930262
[400]	validation_0-auc:0.932251
[500]	validation_0-auc:0.934461
[600]	validation_0-auc:0.93519
[700]	validation_0-auc:0.935153
[800]	validation_0-auc:0.934659
Stopping. Best iteration:
[659]	validation_0-auc:0.93556

[0]	validation_0-auc:0.531029
Will train until validation_0-auc hasn't improved in 200 rounds.
[100]	validation_0-auc:0.907845
[200]	validation_0-auc:0.925699
[300]	validation_0-auc:0.928914
[400]	validation_0-auc:0.934023
[500]	validation_0-auc:

In [ ]:
# Lưu mô hình
MODEL_ROOT_DIR = './xgboost_clf'
if not os.path.exists(MODEL_ROOT_DIR):
    os.mkdir(MODEL_ROOT_DIR)
for i, clf in enumerate(list_clf):
  clf.save_model(f'{MODEL_ROOT_DIR}/xgb_clf{i}.json')

! zip -r xgboost_clf.zip ./xgboost_clf

  adding: xgboost_clf/ (stored 0%)
  adding: xgboost_clf/xgb_clf1.json (deflated 57%)
  adding: xgboost_clf/xgb_clf3.json (deflated 57%)
  adding: xgboost_clf/xgb_clf4.json (deflated 57%)
  adding: xgboost_clf/xgb_clf2.json (deflated 57%)
  adding: xgboost_clf/xgb_clf5.json (deflated 57%)
  adding: xgboost_clf/xgb_clf0.json (deflated 57%)


In [ ]:
# Load mô hình
list_clf = []
MODEL_ROOT_DIR = './xgboost_clf'
for model_path in glob.glob(f'{MODEL_ROOT_DIR}/xgb_clf*.json'):
  clf = base_xgb_model()
  clf.load_model(model_path)
  list_clf.append(clf)
  print('load model ', model_path)

load model  ./xgboost_clf/xgb_clf1.json
load model  ./xgboost_clf/xgb_clf3.json
load model  ./xgboost_clf/xgb_clf4.json
load model  ./xgboost_clf/xgb_clf2.json
load model  ./xgboost_clf/xgb_clf5.json
load model  ./xgboost_clf/xgb_clf0.json


In [ ]:
class EnsemClf():
  def __init__(self, list_clf):
    self.list_clf = list_clf
    pass

  def predict_proba(self, X):
    list_proba = [ clf.predict_proba(X) for clf in self.list_clf ] # [n_clf * n_X * 2]
    list_proba = np.array([ [xprob[1] for xprob in res_clf] for res_clf in list_proba ]) # [n_clf * n_X]
    list_proba = list_proba.mean(axis = 0)
    return list_proba

ensem_clf = EnsemClf(list_clf)
# ensem_clf = list_clf[0]

In [ ]:
ensem_clf.predict_proba(X[:5]).shape

(5,)

# Chọn threshold bằng tập dev

In [ ]:
esp = 1e-10


def precision(ground_truth, predicts):
    true_pos = np.sum([int(i in ground_truth) for i in predicts])
    return true_pos / (len(predicts) + esp)

def recall(ground_truth, predicts):
    true_pos = np.sum([int(i in ground_truth) for i in predicts])
    return true_pos / (len(ground_truth) + esp)

def f2i(ground_truth, predicts):
    pre = precision(ground_truth, predicts)
    rec = recall(ground_truth, predicts)
    return (5 * pre * rec) / (4 * pre + rec + esp)

In [ ]:
top_n = 200
ground_truth = []
prob_list = []
threshold = 0.01

for qid in tqdm(idx_dev):
    relevance_aid = [aid for (qidx, aid) in relevance_pair if qidx == qid]
    ground_truth.append(relevance_aid)
    topn_ranking = get_ranking_result(top_n, qid)[-top_n:]
    # print(len(topn_ranking))
    X = np.array([
            np.concatenate((get_embed_question(qid), get_embed_article(aid),  calculate_tfidf_feature(qid, aid)))
            for aid in topn_ranking
    ])

    pred = ensem_clf.predict_proba(X)
    prob_list.append(pred)

    predict_aid = [aid for i, aid in enumerate(topn_ranking) if pred[i] >= threshold]
    print(predict_aid, relevance_aid)

  0%|          | 0/511 [00:00<?, ?it/s]

[51726, 51660, 51730, 51664] [51730]
[41329, 40287, 41338, 56215, 40281, 40288, 40283, 40286] [40288]
[10389] [10389]
[34001, 33957] [33998]
[15738] [15745]
[] [26571]
[] [27701, 33231]
[22696, 22684, 39426, 22691, 39424] [39424]
[30960] [30960]
[24673, 24678, 24671, 24679] [24673]
[43835, 43838] [43835]
[] [20643]
[56182, 41313, 32325, 27503, 19794, 19414, 17320, 36091] [36091]
[] [47042]
[] [43000]
[10304, 10303, 10314, 10315, 10317] [10314]
[] [47566, 47569]
[39838, 39837] [39837]
[41429] [41522]
[15522, 15526, 38164, 9389, 38168, 15524, 15519, 9390] [15524]
[10304, 10303] [10303]
[] [53608]
[34620] [34620]
[6712] [6712]
[] [10814]
[32491, 32512, 32510] [32510]
[10303, 10304, 10310] [10304]
[22539] [22556]
[15678, 34477, 36538, 36537] [36536]
[10315, 10304, 10303, 10318, 10332, 10308] [10308]
[19213, 19210, 48667, 19221, 19232] [19222]
[] [10107]
[13245] [13245]
[9086, 9062] [9062]
[56126] [56124]
[29870, 48232, 48239, 35762, 35761] [29869]
[] [37373]
[6578, 700] [700]
[3765] [3761]

In [ ]:
top_n = 200

# ground_truth = []
# prob_list = []


for threshold in np.arange(0.1, 0.2, 0.0001):
    print('Threshold = ', threshold)
    total_f2i = 0
    total_prec = 0
    for idx, qid in enumerate(idx_dev):
        topn_ranking = get_ranking_result(top_n, qid)[-top_n:]
        # print(len(topn_ranking))
        # pro = [p[1] for p in prob_list[idx]]
        pro = prob_list[idx]
        # predict_aid = [aid for i, aid in enumerate(topn_ranking) if prob_list[idx][i] >= threshold]
        predict_aid = [aid for i, aid in enumerate(topn_ranking) if pro[i] >= threshold]
        if len(predict_aid) == 0:
            predict_aid = topn_ranking[np.argsort(pro)[-2:]]
        
        f2i_score = f2i(ground_truth[idx], predict_aid)
        total_f2i += f2i_score
        total_prec += precision(ground_truth[idx], predict_aid)

    print('avg_f2i = ', total_f2i / len(ground_truth))
    print('avg_precision = ', total_prec / len(ground_truth))
    print('=' * 20)

In [ ]:
# ques

In [ ]:
top_n = 200

# ground_truth = []
# prob_list = []

threshold = 0.16
total_f2i = 0
total_prec = 0
for idx, qid in enumerate(idx_dev):
    topn_ranking = get_ranking_result(top_n, qid)[-top_n:]
    # print(len(topn_ranking))
    # pro = [p[1] for p in prob_list[idx]]
    pro = prob_list[idx]
    # predict_aid = [aid for i, aid in enumerate(topn_ranking) if prob_list[idx][i] >= threshold]
    predict_aid = [aid for i, aid in enumerate(topn_ranking) if pro[i] >= threshold]
    if len(predict_aid) == 0:
        predict_aid = topn_ranking[np.argsort(pro)[-2:]]
    
    f2i_score = f2i(ground_truth[idx], predict_aid)
    if f2i_score < 0.5:
        print(f2i_score)
        print(predict_aid, relevance_aid)
        print(ques[qid])
        print('probability for grountruth', [p for idp, p in enumerate(pro) if topn_ranking[idp] in ground_truth[idx]])
        print('predict articles: ')
        print(*[articles[aid] for aid in predict_aid])
        print('ground truth articles: ')
        print([articles[aid] for aid in ground_truth[idx]])
    total_f2i += f2i_score
    total_prec += precision(ground_truth[idx], predict_aid)

print('avg_f2i = ', total_f2i / len(ground_truth))
print('avg_precision = ', total_prec / len(ground_truth))
print('=' * 20)

0.0
[51660 51726] [1737]
{'question_id': '7b2530e38d1ebf20153df16af5fb5c76', 'question': 'Mức phạt đối với hành vi không thực hiện cấp phát hàng dự trữ quốc gia được quy định như thế nào?', 'relevant_articles': [{'law_id': '63/2019/nđ-cp', 'article_id': '44'}]}
probability for grountruth [0.038790014]
predict articles: 
{'article_id': '40', 'title': 'Điều 40. Hành vi vi phạm quy định về mua, bán hàng dự trữ quốc gia', 'text': '1. Phạt tiền từ 1.000.000 đồng đến 3.000.000 đồng đối với hành vi không thông báo hoặc niêm yết công khai về đối tượng tham gia mua, bán; phương thức mua, bán; giá mua, bán; thời hạn mua, bán; số lượng, chủng loại, quy cách, tiêu chuẩn, chất lượng hàng dự trữ quốc gia theo quy định của cơ quan nhà nước có thẩm quyền.\n2. Phạt tiền từ 3.000.000 đồng đến 5.000.000 đồng đối với hành vi:\nThực hiện không đúng các quy định do cơ quan nhà nước có thẩm quyền ban hành về phương thức mua, bán đối với từng loại hàng dự trữ quốc gia.\n3. Phạt tiền từ 5.000.000 đồng đến 7.00

# Test với tập test

In [ ]:
top_n = 200
threshold = 0.1
total_f2i = 0


ground_truth = []
prob_list = []


for qid in tqdm(idx_test):
    relevance_aid = [aid for (qidx, aid) in relevance_pair if qidx == qid]
    ground_truth.append(relevance_aid)
    topn_ranking = get_ranking_result(top_n, qid)
    X = np.array([
            np.concatenate((get_embed_question(qid), get_embed_article(aid), calculate_tfidf_feature(qid, aid)))
            for aid in topn_ranking
    ])
    pred = ensem_clf.predict_proba(X)
    prob_list.append(pred)

  0%|          | 0/640 [00:00<?, ?it/s]

In [ ]:
top_n = 200
total_f2i = 0

for threshold in np.arange(0, 1, 0.01):
    print('Threshold = ', threshold)
    total_f2i = 0
    for idx, qid in enumerate(idx_test):
        topn_ranking = get_ranking_result(top_n, qid)
        pro = prob_list[idx]
        predict_aid = [aid for i, aid in enumerate(topn_ranking) if pro[i] >= threshold]
        if len(predict_aid) == 0:
            predict_aid = topn_ranking[np.argsort(pro)[-2:]]
        f2i_score = f2i(ground_truth[idx], predict_aid)
        total_f2i += f2i_score

    print('avg_f2i = ', total_f2i / len(ground_truth))
    print('=' * 20)

Threshold =  0.0
avg_f2i =  0.023481771720287613
Threshold =  0.01
avg_f2i =  0.4438491264220767
Threshold =  0.02
avg_f2i =  0.47116204505848513
Threshold =  0.03
avg_f2i =  0.4739064411710786
Threshold =  0.04
avg_f2i =  0.48571924584750564
Threshold =  0.05
avg_f2i =  0.4895373939723136
Threshold =  0.06
avg_f2i =  0.4921227968503067
Threshold =  0.07
avg_f2i =  0.49166365312942084
Threshold =  0.08
avg_f2i =  0.4891865945979717
Threshold =  0.09
avg_f2i =  0.49010115312960006
Threshold =  0.1
avg_f2i =  0.4889783176318976
Threshold =  0.11
avg_f2i =  0.49112055473481275
Threshold =  0.12
avg_f2i =  0.49043541088568166
Threshold =  0.13
avg_f2i =  0.49030520255238724
Threshold =  0.14
avg_f2i =  0.4900230844968765
Threshold =  0.15
avg_f2i =  0.49097794560786856
Threshold =  0.16
avg_f2i =  0.4904943146555568
Threshold =  0.17
avg_f2i =  0.4889318146557443
Threshold =  0.18
avg_f2i =  0.4883365765605886
Threshold =  0.19
avg_f2i =  0.4876669337035133
Threshold =  0.2
avg_f2i =  0.48

In [ ]:

# best_threshold = 0.21
best_threshold = 0.08
top_n = 200
total_f2i = 0

for qid in tqdm(idx_test):
    
    relevance_aid = [aid for (qidx, aid) in relevance_pair if qidx == qid]
   
    topn_ranking = get_ranking_result(top_n, qid)
    X = np.array([
            np.concatenate((get_embed_question(qid), get_embed_article(aid)))
            for aid in topn_ranking
    ])
    pred = ensem_clf.predict_proba(X)
    predict_aid = [aid for i, aid in enumerate(topn_ranking) if pred[i] >= best_threshold]
    
    print(predict_aid, relevance_aid)
    total_f2i += f2i(relevance_aid, predict_aid)

    # print('Question: ', end=' ')
    # display_segmented_corpus(preproc_stage1_ques[qid])
    # print('-'*10)
    # print('Relevance aid: ')
    # [display_segmented_corpus(preproc_stage1[aid]) for aid in relevance_aid]
    # print('-' * 10)
    # print('Predict Aid')
    # [display_segmented_corpus(preproc_stage1[aid]) for aid in predict_aid]

    print('='*20)

In [ ]:
total_f2i / len(idx_test)

0.097730654750214